# Abstract

Read through a folder, importing everything, to do some basic searching about.

# Environment

In [ ]:
from copy import copy
from importlib import import_module
from inspect import (getmembers, isclass)
import logging
from os import (listdir, path, walk)
import re
import sys

In [ ]:
from jwst.stpipe import Step

In [ ]:
SUFFIXES_TO_DISCARD = set(('functionwrapper', 'systemcall'))

In [ ]:
SUFFIXES_TO_ADD = set((
    'cal', 'calints', 'crf', 'crfints',
    'dark',
    'i2d',
    'jump',
    'psfalign', 'psfstack', 'psfsub',
    'ramp', 'rate', 'rateints',
    's2d', 's3d',
    'uncal',
    'wfscmb',
    'x1d', 'x1dints',
))

# Library

In [ ]:
def folder_traverse(folder_path, basename_regex='.+', path_exclude_regex='^$'):
    """Generator of full file paths for all files
    in a folder.
    
    Parameters
    ----------
    folder_path: str
        The folder to traverse
        
    basename_regex: str
        Regular expression that must match
        the `basename` part of the file path.
        
    path_exclude_regex: str
        Regular expression to exclude a path.
        
    Returns
    -------
    generator
        A generator, return the next file.
    """
    basename_regex = re.compile(basename_regex)
    path_exclude_regex = re.compile(path_exclude_regex)
    for root, dirs, files in walk(folder_path):
        if path_exclude_regex.search(root):
            continue
        for file in files:
            if basename_regex.match(file):
                yield path.join(root, file)

In [ ]:
def load_local_pkg(fpath):
    """Generator producing all modules under fpath
    """
    package_fpath, package = path.split(fpath)
    package_fpath_len = len(package_fpath) + 1
    sys_path = copy(sys.path)
    sys.path.insert(0, package_fpath)
    try:
        for module_fpath in folder_traverse(
            fpath, basename_regex='[^_].+\.py$', path_exclude_regex='tests'
        ):
            folder_path, fname = path.split(module_fpath[package_fpath_len:])
            module_path = folder_path.split('/')
            module_path.append(path.splitext(fname)[0])
            module_path = '.'.join(module_path)
            try:
                module = import_module(module_path)
            except Exception:
                logging.warning('Cannot load module "{}"'.format(module_path))
            else:
                yield module
    except Exception as exception:
        logging.warning('Exception occurred: "{}'.format(exception))
    finally:
        sys.path = sys_path

In [ ]:
def find_suffixes():
    """Find all possible suffixes from the jwst package
    
    Returns
    -------
    suffixes: set
        The set of all programmatically findable suffixes.
        
    Notes
    -----
    This will load all of the `jwst` package. Consider if this
    is worth doing dynamically or only as a utility to update
    a static list.
    """
    suffixes = set()
    
    jwst = import_module('jwst')
    jwst_fpath = path.split(jwst.__file__)[0]
    
    # First traverse the code base and find all
    # `Step` classes. The default suffix is the
    # class name.
    for module in load_local_pkg(jwst_fpath):
        for klass_name, klass in getmembers(
            module,
            lambda o: isclass(o) and issubclass(o, Step)
        ):
            suffixes.add(klass_name.lower())
            
    # Instantiate Steps/Pipelines from their configuration files.
    # Different names and suffixes can be defined in this way.
    # Note: Based on the `collect_pipeline_cfgs` script
    config_path = path.join(jwst_fpath, 'pipeline') 
    for config_file in listdir(config_path):
        if config_file.endswith('.cfg'):
            try:
                step = Step.from_config_file(path.join(config_path, config_file))
            except Exception as exception:
                pass
            else:
                suffixes.add(step.name.lower())
                if step.suffix is not None:
                    suffixes.add(step.suffix.lower())
                    
    # Discard known bad finds.
    suffixes.difference_update(SUFFIXES_TO_DISCARD)
    
    # Add defined suffixes
    suffixes.update(SUFFIXES_TO_ADD)
                    
    # That's all folks
    return suffixes

# Main

In [ ]:
re.search('^$', '/Users/eisenham/Documents/ssbdev/jwst/jwst/associations/notebooks')

In [ ]:
jwst = import_module('jwst')
jwst.__file__

## Setup the all-in-one function

In [ ]:
suffixes_from_find = find_suffixes()

In [ ]:
len(suffixes_from_find)

In [ ]:
suffixes_from_find

## The piecemeal developement

In [ ]:
package_location = '../../../../jwst'

In [ ]:
%ls $package_location

## Start collecting all possible suffixes

In [ ]:
suffixes = set()

## First, find all Step classes
`Step` class names are the default suffix for any `Step`

In [ ]:
modules = list(load_local_pkg(package_location))

In [ ]:
step_klasses = set(
    klass
    for module in modules
    for klass_name, klass in getmembers(module, lambda o: isclass(o) and issubclass(o, Step))
)

In [ ]:
step_klass_objs = []
for klass in step_klasses:
    try:
        klass_obj = klass()
    except Exception as exception:
        logging.warning('Cannot instantiate {}'.format(klass))
    else:
        step_klass_objs.append(klass_obj)


In [ ]:
for step_klass_obj in step_klass_objs:
    try:
        name = step_klass_obj.name
    except Exception as exception:
        pass
    else:
        suffixes.add(name.lower())

In [ ]:
len(suffixes)

## Instantiate from config files
`Step` and `Pipeline` can be instantiated from config files. These can define different names and suffixes. Collect what we can.

_Note_: Based on `collect_cfgs` script where configuration files are only searched for in the `pipeline` module.

In [ ]:
config_path = path.join(package_location, 'pipeline')

In [ ]:
for config_file in listdir(config_path):
    if config_file.endswith('.cfg'):
        try:
            step = Step.from_config_file(path.join(config_path, config_file))
        except Exception as exception:
            logging.warning('Could not instantiate {}'.format(config_file))
        else:
            suffixes.add(step.name.lower())
            if step.suffix is not None:
                suffixes.add(step.suffix.lower())

In [ ]:
len(suffixes)

In [ ]:
suffixes_from_find.symmetric_difference(suffixes)

In [ ]:
suffixes